# Alternating Least Squares (ALS) Matrix Factorization

- Load and validate the **subset_ratings.csv** file  
- Build an **item × user CSR matrix** (ratings as confidence)  
- Fit an **explicit‑feedback ALS** model (latent factors, regularization, 15 iter)  
- Map internal indices back to original `movieId` / `userId` for results  
- Split data per user (80 % train / 20 % test) and report **RMSE / MAE**  
- Output **Top‑N recommendations** for 1 000 sampled users to JSON

In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from implicit.als import AlternatingLeastSquares
from pathlib import Path
import json, random
from tqdm import tqdm

In [2]:
ratings = pd.read_csv("subset_ratings.csv", usecols=["userId", "movieId", "rating"])
print("Loaded:", ratings.shape)

Loaded: (2078625, 3)


In [3]:
def split_per_user(df, test_frac=0.2, seed=7):
    rng = np.random.default_rng(seed)
    train, test = [], []
    for _, grp in df.groupby("userId"):
        if len(grp) < 2:
            train.append(grp)
        else:
            mask = rng.random(len(grp)) < (1 - test_frac)
            train.append(grp[mask])
            test.append(grp[~mask])
    return pd.concat(train), pd.concat(test)

train_df, test_df = split_per_user(ratings)
print("train / test:", train_df.shape, test_df.shape)

train / test: (1664041, 3) (414584, 3)


In [4]:
user_ids = train_df.userId.unique()       
item_ids_raw = train_df.movieId.unique()  

uid_map = {u: i for i, u in enumerate(user_ids)}
mid_map = {m: i for i, m in enumerate(item_ids_raw)}

idx2mid = np.empty(len(item_ids_raw), dtype=np.int32)
for mid, idx in mid_map.items():
    idx2mid[idx] = mid


def df_to_csr(df):
    rows = df.movieId.map(mid_map)
    cols = df.userId.map(uid_map)
    data = df.rating.astype(np.float32)
    shape = (len(item_ids_raw), len(user_ids))
    return sparse.csr_matrix((data, (rows, cols)), shape=shape)

train_mat = df_to_csr(train_df)
print("CSR shape (items×users):", train_mat.shape)

CSR shape (items×users): (7049, 10000)


In [5]:
als = AlternatingLeastSquares(
    factors=64,
    regularization=0.1,
    iterations=15,
    use_native=True,
    dtype=np.float32,
)
als.fit(train_mat)

d:\Python\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 20 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

In [6]:
uid_test_idx = test_df.userId.map(uid_map)
mid_test_idx = test_df.movieId.map(lambda m: mid_map.get(m, -1))
mask = mid_test_idx >= 0

uid_vec = uid_test_idx.values[mask]
mid_vec = mid_test_idx.values[mask]

mask2 = uid_vec < als.user_factors.shape[0]
uid_vec, mid_vec = uid_vec[mask2], mid_vec[mask2]
y_true = test_df.rating.values[mask][mask2]

y_pred = np.sum(als.user_factors[uid_vec] * als.item_factors[mid_vec], axis=1)
rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
mae  = np.mean(np.abs(y_true - y_pred))
print(f"ALS  RMSE = {rmse:.4f}   MAE = {mae:.4f}")

ALS  RMSE = 3.6305   MAE = 3.4753


In [7]:
user_items_full = train_mat.T.tocsr() 

K, N = 10, 1_000
sample_users = random.sample(list(user_ids), min(N, len(user_ids)))

preds = {}
for u in tqdm(sample_users, desc=f"Top‑{K} recs"):
    uid = uid_map.get(u)
    if uid is None or uid >= als.user_factors.shape[0]:
        continue 

    rec_iids, _ = als.recommend(
        userid=uid,
        user_items=user_items_full[uid],  # 1‑row CSR
        N=K,
        filter_already_liked_items=True,
    )

    valid_rec_iids = [i for i in rec_iids if i < idx2mid.shape[0] and idx2mid[i] != -1]
    preds[int(u)]  = [int(idx2mid[i]) for i in valid_rec_iids]

Path("predictions").mkdir(exist_ok=True)
with open("predictions/als_top10_subset.json", "w") as fp:
    json.dump(preds, fp)

print(f"✅  Saved {len(preds)} users → predictions/als_top10_subset.json")

Top‑10 recs: 100%|██████████| 1000/1000 [00:00<00:00, 6903.20it/s]

✅  Saved 690 users → predictions/als_top10_subset.json


# Cold start

In [8]:
scenario = "STANDARD"   # or "USER" or "ITEM"


In [9]:
if scenario=="STANDARD":
    ratings  = pd.read_csv("subset_ratings.csv", usecols=["userId","movieId","rating"])
    train_df, test_df = split_per_user(ratings)
elif scenario=="USER":
    train_df = pd.read_csv("evaluation/user_cold_train.csv", usecols=["userId","movieId","rating"])
    test_df  = pd.read_csv("evaluation/user_cold_test.csv",  usecols=["userId","movieId","rating"])
else:  # ITEM
    train_df = pd.read_csv("evaluation/item_cold_train.csv", usecols=["userId","movieId","rating"])
    test_df  = pd.read_csv("evaluation/item_cold_test.csv",  usecols=["userId","movieId","rating"])

In [10]:
# ——————————————————————————————————————————————————————————————
# Generate Top-K recs for every user in test_df, using internal IDs
# ——————————————————————————————————————————————————————————————
K = 10

# decide test users
test_users = test_df['userId'].unique().tolist()

# prepare the full user_items CSR (items×users → transpose to users×items)
user_items_full = train_mat.T.tocsr()

preds = {}
for u in tqdm(test_users, desc=f"Top-{K} ALS recs ({scenario})"):
    # look up internal user index
    inner_u = uid_map.get(u, None)
    if inner_u is None or inner_u >= als.user_factors.shape[0]:
        # unseen in training, skip or assign empty
        continue

    # now pass the internal index to .recommend
    rec_iids, _ = als.recommend(
        userid=inner_u,
        user_items=user_items_full[inner_u],
        N=K,
        filter_already_liked_items=True,
    )

    # map back to raw movieId
    raw_recs = [ int(idx2mid[i]) for i in rec_iids if i < len(idx2mid) ]
    preds[int(u)] = raw_recs

# dump to JSON
outfn = Path("coldstart_pred") / f"als_{scenario.lower()}_top{K}.json"
outfn.parent.mkdir(exist_ok=True)
with open(outfn, "w") as fp:
    json.dump(preds, fp, indent=2)

print(f"✅  Saved {len(preds)} users → {outfn}")


Top-10 ALS recs (STANDARD): 100%|██████████| 10000/10000 [00:01<00:00, 6405.68it/s]

✅  Saved 7049 users → coldstart_pred\als_standard_top10.json
